In [ ]:
!pip install torch transformers datasets accelerate peft bitsandbytes


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/.../dataset_half_balanced.json")

In [ ]:
from huggingface_hub import login
login(token='hf_your_token_here')


In [ ]:
from transformers import AutoTokenizer

# Carica il tokenizer pre-addestrato del modello Llama 2 7B in formato Hugging Face.
# Il tokenizer serve a convertire testo in token numerici comprensibili dal modello.
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Imposta il token di padding uguale al token di fine sequenza (EOS token).
# Questo è utile perché alcuni modelli non hanno un token di padding dedicato.
tokenizer.pad_token = tokenizer.eos_token


# Funzione per tokenizzare un esempio del dataset
def tokenize(example):
    # Crea un prompt strutturato a partire dai campi del dataset
    # 'instruction', 'input', 'output'. Il prompt ha la forma:
    # ### Instruction:
    # <istruzione>
    # ### Input:
    # <input>
    # ### Response:
    # <output>
    # Questo formato è spesso usato per addestrare modelli instruction-following.
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    
    # Tokenizza il prompt:
    # - truncation=True → tronca il testo se supera max_length
    # - padding="max_length" → aggiunge padding fino a max_length
    # - max_length=512 → lunghezza massima dei token
    # Restituisce un dizionario con input_ids e attention_mask pronto per il modello.
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)


# Applica la funzione di tokenizzazione a tutto il dataset.
# `dataset.map()` crea un nuovo dataset in cui ogni esempio è già tokenizzato.
tokenized_dataset = dataset.map(tokenize)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# Carica il modello Llama-2 7B pre-addestrato in modalità Causal Language Modeling.
# load_in_8bit=True → utilizza la quantizzazione a 8 bit per ridurre l'uso di memoria.
# device_map="auto" → assegna automaticamente i layer del modello ai dispositivi disponibili (CPU/GPU).
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_8bit=True,
    device_map="auto"
)

# Configurazione per il LoRA (Low-Rank Adaptation)
# LoRA permette di fare fine-tuning aggiungendo pochi parametri senza aggiornare tutto il modello.
peft_config = LoraConfig(
    r=8,                        # Rank della matrice di aggiornamento low-rank
    lora_alpha=32,               # Moltiplicatore di scaling per stabilizzare l’addestramento LoRA
    target_modules=["q_proj", "v_proj"],  # Layer del modello dove applicare LoRA (tipicamente Q e V delle attention)
    lora_dropout=0.05,           # Dropout applicato ai pesi LoRA per regolarizzazione
    bias="none",                 # Non aggiunge bias aggiuntivo nel fine-tuning
    task_type=TaskType.CAUSAL_LM # Tipo di task: Causal Language Modeling
)

# Applica LoRA al modello originale.
# Restituisce un modello PEFT che contiene i pesi originali congelati + i parametri LoRA addestrabili.
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Configurazione dei parametri di training
training_args = TrainingArguments(
    output_dir="./llama-finetuned",  # Cartella dove salvare i checkpoint del modello fine-tuned
    per_device_train_batch_size=2,    # Dimensione del batch per GPU/TPU/CPU
    gradient_accumulation_steps=4,    # Accumula i gradienti per simulare batch più grandi
    num_train_epochs=1,               # Numero di epoche di training sul dataset
    learning_rate=2e-4,               # Learning rate per l'ottimizzatore
    logging_dir="./logs",             # Cartella per salvare i log di training
    logging_steps=10,                 # Frequenza (in step) di scrittura dei log
    save_strategy="epoch",            # Salva il modello alla fine di ogni epoca
    fp16=True                         # Abilita mixed precision (half precision) per ridurre uso memoria e velocizzare training
)

# Collator dei dati per il Language Modeling
# Prepara batch di input per il modello
# mlm=False → modello non usa masked language modeling, adatto a causal LM come Llama
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Creazione del Trainer di Hugging Face
trainer = Trainer(
    model=model,                      # Modello da allenare
    args=training_args,                # Parametri di training definiti sopra
    train_dataset=tokenized_dataset["train"],  # Dataset di training già tokenizzato
    tokenizer=tokenizer,               # Tokenizer associato al modello
    data_collator=data_collator        # Funzione che prepara i batch durante il training
)

# Avvia il training del modello
trainer.train()


In [ ]:
prompt = """### Instruction:
You are an email security analysis assistant
### Input:
We attempted to deliver your package today, but were unable to complete the delivery due to missing address information.
Please update your delivery details as soon as possible to avoid return of the shipment:
Update Delivery Information
Thank you for your cooperation,
Logistics Service Team
### Response:
"""

enc = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
)

input_ids = enc.input_ids.cuda()
attention_mask = enc.attention_mask.cuda()

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
   

In [ ]:
model.save_pretrained("./llama-finetuned")
tokenizer.save_pretrained("./llama-finetuned")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ----------------------------
# CONFIGURAZIONE PATH E MODELLO
# ----------------------------

BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
LORA_ADAPTER_PATH = "./llama-finetuned"

# ----------------------------
# CARICAMENTO TOKENIZER
# ----------------------------

tokenizer = AutoTokenizer.from_pretrained(LORA_ADAPTER_PATH)

# Per LLaMA è buona pratica assicurarsi che il pad_token sia definito
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ----------------------------
# CARICAMENTO MODELLO BASE
# ----------------------------

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    load_in_8bit=True,        # oppure load_in_4bit=True se usi QLoRA
    device_map="auto"
)

# ----------------------------
# CARICAMENTO ADAPTER LoRA
# ----------------------------

model = PeftModel.from_pretrained(
    base_model,
    LORA_ADAPTER_PATH
)

model.eval()



In [ ]:
prompt = """### Instruction:
You are an email security analysis assistant
### Input:
| As far as I'm concerned this is only a minor irritant -- Caps Lock is
| pointless anyway in these days of OPERATING SYSTEMS THAT DON'T REQUIRE
| YOU TO SHOUT -- but I wondered if anyone else had noticed this bug-ette
| and/or had a fix for it?

It doesn't show up here, running MIT X11.  

In terms of shouting, if you use MODULA-3, encrusted as it is with
upper case keywords, caps-lock is about the only alternative (and a
poor one at that) to a context sensitive editor like emacs.

### Response:
"""

enc = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
)

input_ids = enc.input_ids.cuda()
attention_mask = enc.attention_mask.cuda()

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
   